In [1]:
import torch
import torch.nn as nn
import torchvision

In [2]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5), (0.5))
])

In [3]:
l = [['a','b'],2,3,4,5]
i = iter(l)

a,b  = next(i)
print(a, b)
a = next(i)
print(a)

a b
2


In [4]:
import torch.utils.data.dataloader

BATCH_SIZE = 64

trainset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    transform=transform,
    download=True
)

trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

print(trainloader)



testset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    transform=transform,
    download=True
)

testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

100%|██████████| 170M/170M [00:05<00:00, 33.1MB/s] 


In [5]:
#build NN model
import torch.nn.functional as F

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(3 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64,10)
    
    def forward(self, x):
        x = x.view(-1, 3 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = NN()
print(model)
        

NN(
  (fc1): Linear(in_features=3072, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)


In [6]:
iterater = iter(trainloader)
images, labels = next(iterater)
print(images.shape)
print(f"label: {labels}")
output = model(images)
print(output.shape)

torch.Size([64, 3, 32, 32])
label: tensor([9, 4, 5, 3, 7, 9, 7, 1, 0, 5, 8, 6, 3, 7, 4, 4, 6, 0, 6, 6, 6, 4, 7, 6,
        9, 9, 0, 6, 6, 7, 9, 1, 0, 1, 5, 6, 4, 1, 6, 2, 7, 7, 6, 5, 2, 9, 0, 2,
        3, 6, 3, 2, 4, 1, 4, 5, 2, 6, 1, 5, 6, 2, 2, 7])
torch.Size([64, 10])


In [7]:
_, predicted = torch.max(output, 1)  # 最も大きいスコアのインデックスを取得
print("Predicted labels:", predicted)

Predicted labels: tensor([1, 1, 1, 1, 1, 2, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 3,
        1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1])


In [8]:

import matplotlib.pyplot as plt
import numpy as np

# CIFAR-10 のクラスラベル
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# 画像を表示する関数
def imshow(img):
    img = img / 2 + 0.5  # 正規化を戻す
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# 画像を表示
#imshow(torchvision.utils.make_grid(images))

# 予測結果を表示
#print('Predicted:', ' '.join(f'{classes[predicted[j]]:5s}' for j in range(4)))


In [9]:
iscorrect = predicted == labels
print(iscorrect)

count_correct = 0
for i in range(BATCH_SIZE):
    if predicted[i] == labels[i]:
        count_correct += 1

print(f"The number of CORRECT ANSWERS: {count_correct}")
print(f"The rate of accuracy: {count_correct/BATCH_SIZE}")

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False, False,  True, False,  True,
        False, False, False, False, False, False, False, False, False, False,
         True, False, False,  True, False, False, False, False,  True, False,
        False, False, False, False])
The number of CORRECT ANSWERS: 6
The rate of accuracy: 0.09375


In [10]:
#after fix

count_correct = iscorrect.sum().item()
print(f"The number of CORRECT ANSWERS: {count_correct}")
print(f"The rate of accuracy: {count_correct/labels.size(0)}")

The number of CORRECT ANSWERS: 6
The rate of accuracy: 0.09375


In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr = 0.0001)

In [12]:
num_epochs = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, label in trainloader:
        
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(output, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    epoch_loss = running_loss / len(trainloader)
    epoch_acc = correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.4%}")

print("finished training")
        

ValueError: Expected input batch_size (16) to match target batch_size (64).